<H2>Exercise: SQL Queries and Data Imputation</H3>

In this exercise you will get acquainted with SQL and data imputation techniques.  The dataset for this exercise includes economic data on various countries (based on UN data but modified). It is attached to this exercise 
Please submit:
(1)	 A file called answers.pdf including the answers to questions: 
-	In part 1:    Two queries and their output 
-	In part 2: -  An explanation for your choice of imputation
                  - The output of the two queries run on the data 
                     after imputation
-	In part 3: - the decomposed schema
                       - The two queries, rewritten for the 
                          decomposed schema
(2)	 Your Python Code

<H3>Part 0:</H3> Load the data into a pandas dataframe, using 
import pandas as pd
data=pd.read_csv(filename)

In [2]:
import duckdb
import pandas as pd

In [3]:
data = pd.read_csv('poverty_raw_data.csv')
data.head()

,record_id,region_code,country_name,country_code,survey_acronym,reporting_year,reporting_level,survey_coverage,survey_year,welfare_type,...,decile9,decile10,cpi,ppp,reporting_pop,reporting_gdp,reporting_pce,is_interpolated,distribution_type,estimation_type
0,1,NaN,Angola,AGO,HBS,2000,national,national,2000.21,consumption,...,0.158687,0.402455,0.014200,107.269989,16394062,1840.353828,NaN,0,micro,survey
1,2,SSA,Angola,AGO,IBEP-MICS,2008,national,national,2008.50,consumption,...,0.162333,0.323117,0.303449,107.269989,21691522,3061.636272,1216.312411,0,micro,survey
2,3,SSA,Angola,AGO,IDREA,2018,national,national,2018.17,consumption,...,0.159390,0.396244,1.231024,107.269989,31273533,2677.249833,1548.153535,0,micro,survey
3,4,ECA,Armenia,ARM,ILCS,2012,national,national,2012.00,consumption,...,0.141960,0.245402,0.888864,167.312210,2914421,3278.670451,2953.453311,0,micro,survey
4,5,ECA,Albania,ALB,EWS,1996,national,national,1996.00,consumption,...,0.148995,0.212156,0.399635,50.357372,3168033,1633.552004,1716.336806,0,micro,survey


In [24]:
data.columns

Index(['record_id', 'region_code', 'country_name', 'country_code',
       'survey_acronym', 'reporting_year', 'reporting_level',
       'survey_coverage', 'survey_year', 'welfare_type',
       'survey_comparability', 'comparable_spell', 'poverty_line', 'headcount',
       'poverty_gap', 'poverty_severity', 'watts', 'mean', 'median', 'mld',
       'gini', 'polarization', 'decile1', 'decile2', 'decile3', 'decile4',
       'decile5', 'decile6', 'decile7', 'decile8', 'decile9', 'decile10',
       'cpi', 'ppp', 'reporting_pop', 'reporting_gdp', 'reporting_pce',
       'is_interpolated', 'distribution_type', 'estimation_type'],
      dtype='object')

<H3>Part 1: SQL Queries on the raw data</H3>

In this part you will write two queries and run them on the dataset, using a python library of your choice. A simple way is through duckdb that can be used to execute an SQL query (given as a string) directly on dataframes. 
1)	Write an SQL query that finds all the countries in Latin America and the Caribbean (region code LAC) for which there was a national survey (according to attribute survey_coverage), and return the country name, survey acronym and year. Avoid duplicated rows in the result. 

Report the query, sample 5 rows from the result, and the total number of rows in the result.

2)	Write an SQL query that finds the country (or countries, if there is a tie) in Sub-Saharan Africa (region code SSA) having the lowest average Personal Consumption Expenditures (attribute name reporting_pce). The query result should include the country name and its average PCE.

Report the query and its result.

In [43]:
query_LAC = '''SELECT country_name, survey_acronym, reporting_year
                        FROM data
                        WHERE region_code='LAC' and survey_coverage='national'
                        '''

data1 = duckdb.query(query_LAC)
data1

┌───────────────┬────────────────┬────────────────┐
│ country_name  │ survey_acronym │ reporting_year │
│    varchar    │    varchar     │     int64      │
├───────────────┼────────────────┼────────────────┤
│ Bolivia       │ EH             │           2013 │
│ Bolivia       │ EH             │           2014 │
│ Bolivia       │ EH             │           2015 │
│ Bolivia       │ EH             │           2017 │
│ Bolivia       │ EH             │           2020 │
│ Bolivia       │ EH             │           2021 │
│ Brazil        │ PNAD           │           1981 │
│ Brazil        │ PNAD           │           1982 │
│ Brazil        │ PNAD           │           1983 │
│ Brazil        │ PNADC-E1       │           2016 │
│   ·           │  ·             │             ·  │
│   ·           │  ·             │             ·  │
│   ·           │  ·             │             ·  │
│ Venezuela, RB │ EHM            │           2001 │
│ Venezuela, RB │ EHM            │           2002 │
│ Venezuela,

In [44]:
#sample 5 rows from the result
query_LAC_lim = '''SELECT country_name, survey_acronym, reporting_year
                        FROM data
                        WHERE region_code='LAC' and survey_coverage='national'
                        LIMIT 5
                        '''
data1_lim = duckdb.query(query_LAC_lim)
data1_lim

┌──────────────┬────────────────┬────────────────┐
│ country_name │ survey_acronym │ reporting_year │
│   varchar    │    varchar     │     int64      │
├──────────────┼────────────────┼────────────────┤
│ Bolivia      │ EH             │           2013 │
│ Bolivia      │ EH             │           2014 │
│ Bolivia      │ EH             │           2015 │
│ Bolivia      │ EH             │           2017 │
│ Bolivia      │ EH             │           2020 │
└──────────────┴────────────────┴────────────────┘

In [45]:
# Write an SQL query that finds the country (or countries, if there is a tie) in Sub-Saharan Africa (region code SSA) 
# having the lowest average Personal Consumption Expenditures (attribute name reporting_pce). 
# The query result should include the country name and its average PCE.
query_SSA = '''WITH tabl AS
                        (SELECT country_name, AVG(reporting_pce) as avg_pce
                        FROM data
                        WHERE region_code='SSA'
                        GROUP BY country_name)

                        SELECT country_name, avg_pce
                        FROM tabl
                        WHERE avg_pce = (
                            SELECT MIN(avg_pce)
                            FROM tabl)
                        '''

data2 = duckdb.query(query_SSA)
data2

┌──────────────────┬──────────────┐
│   country_name   │   avg_pce    │
│     varchar      │    double    │
├──────────────────┼──────────────┤
│ Congo, Dem. Rep. │ 305.26031975 │
└──────────────────┴──────────────┘

In [42]:
data_check = duckdb.query('''SELECT country_name, AVG(reporting_pce) as avg_pce
                        FROM data
                        WHERE region_code='SSA'
                        GROUP BY country_name
                        ORDER BY avg_pce
                        ''')
data_check

┌──────────────────────────┬────────────────────┐
│       country_name       │      avg_pce       │
│         varchar          │       double       │
├──────────────────────────┼────────────────────┤
│ Congo, Dem. Rep.         │       305.26031975 │
│ Mozambique               │ 310.62590721999993 │
│ Niger                    │  319.5865434285715 │
│ Madagascar               │ 375.65859372499995 │
│ Lesotho                  │ 403.91992623249996 │
│ Burkina Faso             │  408.2346282333333 │
│ Rwanda                   │ 420.87566740000005 │
│ Central African Republic │        428.0250348 │
│ Ethiopia                 │        451.2989894 │
│ Mali                     │        463.6845969 │
│  ·                       │             ·      │
│  ·                       │             ·      │
│  ·                       │             ·      │
│ Cabo Verde               │ 1894.3482669999999 │
│ Namibia                  │  2602.759125666667 │
│ South Africa             │  3346.448178166667 │


<H3>Part 2: Data Imputation</H3>

1) The data includes 3 attributes with NULL values in some rows. 
For each such attribute, use an imputation method of your choice, either via sklearn.impute or your own implementation, to impute the NULLs in the dataframe. Different methods can be used for different attributes. Explain your choice of methods.
2) Rerun the queries from part 1 over the dataframe after imputation and compare the results.



In [49]:
# columns with missing values
data.isna().sum()[data.isna().sum()>0]

region_code        102
survey_coverage     74
reporting_pce      260
dtype: int64

In [60]:
data[[ 'region_code', 'country_name', 'country_code',
       'survey_acronym', 'reporting_year', 'reporting_level',
       'survey_coverage', 'survey_year', 'reporting_pce'
       ]].sort_values(by=['country_name', 'reporting_year']).head(100)

,region_code,country_name,country_code,survey_acronym,reporting_year,reporting_level,survey_coverage,survey_year,reporting_pce
4,ECA,Albania,ALB,EWS,1996,national,national,1996.0,1716.336806
5,ECA,Albania,ALB,LSMS,2002,national,national,2002.0,1685.203083
7,NaN,Albania,ALB,LSMS,2005,national,national,2005.0,2079.203480
9,ECA,Albania,ALB,LSMS,2008,national,national,2008.0,2820.279521
10,ECA,Albania,ALB,LSMS,2012,national,national,2012.0,2990.396606
...,...,...,...,...,...,...,...,...,...
102,NaN,Austria,AUT,ECHP-LIS,2000,national,national,2000.0,21252.013580
104,NaN,Austria,AUT,EU-SILC,2003,national,national,2003.0,21750.471780
107,NaN,Austria,AUT,EU-SILC,2004,national,national,2004.0,22117.628680
110,OHI,Austria,AUT,EU-SILC,2005,national,national,2005.0,22461.382120


After exploring the data, we have decided on the following strategies to fill missing values:

- For the region_code column: Since each country has a similar region code, we will fill the missing values based on values from other rows for the same country.

- For the survey_coverage column: It appears that survey coverage is constant for a country. Therefore, we will apply the same strategy as for the region_code column.

- For the reporting_pce column, we will follow these steps:
    1. Sort the data by country and by year.
    2. If data is available for both the previous and the next year, we will calculate the mean of those closest values and fill the missing value with this result.
    3. If the missing value appears at the beginning or end of the list (the first or last year for a country), we will fill it with the value from the closest year

In [136]:
# create df with country_name as index and two columns 'region_code' and 'survey_coverage'
data_tmp = data[['country_name', 'region_code', 'survey_coverage']].drop_duplicates().dropna()


def change_na_rc(x):
    if pd.isnull(x['region_code']):
        try:
            return data_tmp[data_tmp['country_name'] == x['country_name']]['region_code'].values[0]
        except:
            return 'NOCODE'
    else:
        return x['region_code']

def change_na_sc(x):
    if pd.isnull(x['survey_coverage']):
        return data_tmp[data_tmp['country_name'] == x['country_name']]['survey_coverage'].values[0]
    else:
        return x['survey_coverage']

In [139]:
data_upd = data.copy()
data_upd['region_code'] = data_upd.apply(change_na_rc, axis=1)
data_upd['survey_coverage'] = data_upd.apply(change_na_sc, axis=1)
data_upd[['region_code', 'survey_coverage']].isnull().sum()

region_code        0
survey_coverage    0
dtype: int64

<H3>Part 3: Decomposition</H3>
In this part, we will work with the new data after imputation – use the results of an imputation method of your choice from part 2 (as stated above, different methods can be used for different attributes).
1)	Manually find at least two Functional Dependencies (FDs) in the data (not involving the record_id column; both can involve the same lefthand side). Report the FDs you have found in the pdf file. 

2)	Based on your observed FDs, decompose the table from the dataset according to the FDs into BCNF and design a database schema corresponding to the decomposition. Submit the schema in the pdf file either as a list of relations (along with column names, keys and foreign keys(, or through E/R diagrams.

For simplicity, please discard attributes that are not used in the queries nor in FDs nor in imputation.

3)	Implement and populate the new schema with the data you have obtained after imputation in step 2. Rewrite the queries from part 1 so that they work for the decomposed schema and execute each query. Verify that the results match your results from the part 2.
Submit the queries.  
For the implementation, you can use pandas dataframes again (where you generate a new dataframe for each relation including the projection of the data on some attributes), or any other implementation.